Siamese Network의 핵심 아이디어는 비교 대상이 되는 두 데이터 쌍을 임베딩 벡터로 변환하여, 유사성을 측정하는 것입니다.

Siamese Network은 두 개의 입력 데이터를 동일한 네트워크 아키텍처를 통해 통과시키고, 각각의 입력에 대한 임베딩 벡터를 얻습니다. 이후에는 두 벡터 간의 유사도를 측정하기 위해 일련의 유사도 측정 방법을 사용할 수 있습니다. 예를 들면, 유클리드 거리, 맨해튼 거리, 코사인 유사도 등을 계산하여 두 벡터 간의 유사성을 판단할 수 있습니다.

In [2]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
# 더미데이터 불러오기

df = pd.read_excel('../데이터/더미데이터.xlsx', index_col=0)

for col in range(2):
    df.loc[0:20, col] = df.loc[0:20, col].apply(lambda x: [float(value) for value in x.strip('[ ]').split()])

In [5]:
df

,0,1
0,"[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 60000.0, 0.0, 0.0, 0.0, 0.0]"
1,"[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 67000.0, 0.0, 0.0, 0.0, 0.0]"
2,"[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 60000.0, 0.0, 0.0, 0.0, 0.0]"
3,"[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 72250.0, 0.0, 0.0, 0.0, 0.0]"
4,"[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 73500.0, 0.0, 0.0, 0.0, 0.0]"
5,"[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 76500.0, 0.0, 0.0, 0.0, 0.0]"
6,"[0.0, 0.0, 60000.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 67000.0, 0.0, 0.0, 0.0, 0.0]"
7,"[0.0, 0.0, 60000.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 60000.0, 0.0, 0.0, 0.0, 0.0]"
8,"[0.0, 0.0, 60000.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 72250.0, 0.0, 0.0, 0.0, 0.0]"
9,"[0.0, 0.0, 60000.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 73500.0, 0.0, 0.0, 0.0, 0.0]"


In [9]:
a = np.array(
    [
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]]
    ]
)
print(a.shape)

(3, 2, 7)


In [28]:
# MLP Network
class MultiLayerPerceptronNetwork(nn.Module):
    def __init__(self):
        super(MultiLayerPerceptronNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(7, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 512)
        )
    
    def forward_once(self, x):
        output = self.fc(x)
        return output
    
    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

# 데이터셋
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

data = torch.FloatTensor(df.values.tolist())
dataset = CustomDataset(data)

# 데이터로더
batchsize = 21
dataloader = DataLoader(dataset=dataset, batch_size=batchsize, shuffle=True)

# 인스턴스
model = MultiLayerPerceptronNetwork()

# 손실 함수와 옵티마이저
class ContrastiveLoss(nn.Module):
    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - label) * 0.5 * torch.pow(euclidean_distance, 2) +
                                      label * 0.5 * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive
criterion = ContrastiveLoss(margin=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습
epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for batch_data in dataloader:
        # 기울기 초기화
        optimizer.zero_grad() 

        # 입력 벡터
        input1, input2 = batch_data[:, 0], batch_data[:, 1]

        # 임베딩 벡터
        output1, output2 = model(input1, input2)
        
        # 라벨 벡터
        label = torch.ones_like(output1)
        
        # 유사도 측정 및 손실 계산
        loss = criterion(output1, output2, label)
        
        # 역전파 및 가중치 업데이트
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    # 현재 에포크에서의 손실 출력
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}")

# 학습된 모델을 이용한 임베딩 추출
embedding = model.forward_once(torch.tensor(data).view(-1, 7))
print("Embedding shape:", embedding.shape)

RuntimeError: The size of tensor a (512) must match the size of tensor b (21) at non-singleton dimension 1

In [3]:
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader

class SiameseNetworkDataset(Dataset):

    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return self.data[index][0], self.data[index][1]

    def __len__(self):
        return len(self.data)

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(7, 128),
            nn.ReLU(),
            nn.Linear(128, 64)
        )

    def forward_once(self, x):
        output = self.fc(x)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

a = np.array(
    [
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]],
        [[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 59500.0, 0.0, 0.0, 0.0, 0.0]]
    ]
)
tensor_a = torch.tensor(a, dtype=torch.float32)

dataset = SiameseNetworkDataset(tensor_a)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

model = SiameseNetwork()
optimizer = SGD(model.parameters(), lr=0.01)

def siamese_loss(output1, output2, distance='euclidean'):
    if distance == 'euclidean':
        loss = torch.dist(output1, output2)
    elif distance == 'manhattan':
        loss = torch.abs(output1 - output2).sum()
    elif distance == 'cosine':
        similarity = F.cosine_similarity(output1, output2)
        loss = 1 - similarity
    elif distance == 'jaccard':
        intersection = torch.min(output1, output2).sum()
        union = torch.max(output1, output2).sum()
        similarity = intersection / union
        loss = 1 - similarity
    return loss

for epoch in range(100):  # Training for 100 epochs
    for i, data in enumerate(dataloader):
        input1, input2 = data
        optimizer.zero_grad()
        embedding1, embedding2 = model(input1, input2)
        loss = siamese_loss(embedding1, embedding2, distance='cosine')  # Euclidean Distance
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


Epoch 1, Loss: 1.1920928955078125e-07
Epoch 2, Loss: 1.1920928955078125e-07
Epoch 3, Loss: 5.960464477539063e-08
Epoch 4, Loss: 5.960464477539063e-08
Epoch 5, Loss: 5.960464477539063e-08
Epoch 6, Loss: 5.960464477539063e-08
Epoch 7, Loss: 5.960464477539063e-08
Epoch 8, Loss: 5.960464477539063e-08
Epoch 9, Loss: 5.960464477539063e-08
Epoch 10, Loss: 0.0
Epoch 11, Loss: 0.0
Epoch 12, Loss: 0.0
Epoch 13, Loss: 0.0
Epoch 14, Loss: 0.0
Epoch 15, Loss: 0.0
Epoch 16, Loss: 0.0
Epoch 17, Loss: 0.0
Epoch 18, Loss: 0.0
Epoch 19, Loss: 0.0
Epoch 20, Loss: 0.0
Epoch 21, Loss: 0.0
Epoch 22, Loss: 0.0
Epoch 23, Loss: 0.0
Epoch 24, Loss: 0.0
Epoch 25, Loss: 0.0
Epoch 26, Loss: 0.0
Epoch 27, Loss: 0.0
Epoch 28, Loss: 0.0
Epoch 29, Loss: 0.0
Epoch 30, Loss: 0.0
Epoch 31, Loss: 0.0
Epoch 32, Loss: 0.0
Epoch 33, Loss: 0.0
Epoch 34, Loss: 0.0
Epoch 35, Loss: 0.0
Epoch 36, Loss: 0.0
Epoch 37, Loss: 0.0
Epoch 38, Loss: 0.0
Epoch 39, Loss: 0.0
Epoch 40, Loss: 0.0
Epoch 41, Loss: 0.0
Epoch 42, Loss: 0.0
Epoch